In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 5s (51.2 kB/s)
Reading package li

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-07 19:18:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.6’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.79MB/s    in 0.2s    

2022-08-07 19:18:53 (4.79 MB/s) - ‘postgresql-42.2.9.jar.6’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in jewelry data from S3 Buckets
from pyspark import SparkFiles
url="https://myhwbucket-cle-202208070845.s3.us-west-1.amazonaws.com/amazon_reviews_us_Jewelry_v1_00.tsv" 
spark.sparkContext.addFile(url)
jewelry_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
jewelry_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
# Count the number of records (rows) in the jewelry dataset
jewelry_data_df.count()

1767753

In [6]:
# Read in toys data from S3 Buckets
from pyspark import SparkFiles
url="https://myhwbucket-cle-202208070845.s3.us-west-1.amazonaws.com/amazon_reviews_us_Toys_v1_00.tsv" 
spark.sparkContext.addFile(url)
toys_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
toys_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [7]:
# Count the number of records (rows) in the toys dataset
toys_data_df.count()

4864249

In [8]:
# Combine all records into one dataframe, and display total record count
jewelry_toys_df = jewelry_data_df.union(toys_data_df)
jewelry_toys_df.count()

6632002

In [9]:
# Remove duplicate rows
jewelry_toys_df = jewelry_toys_df.dropDuplicates()
jewelry_toys_df.count()

6632002

In [10]:
# Create user dataframe to match review_id_table
# Select matching features and confirm data types
reviews = jewelry_toys_df.select('review_id','customer_id','product_id','product_parent','review_date')
reviews.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]

In [11]:
# Transform review_date to SQL date type
from pyspark.sql.functions import to_date
reviews_final = reviews.withColumn('review_date',to_date(reviews.review_date, 'yyyy-MM-dd'))
reviews_final.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [12]:
# Create user dataframe to match products table
# Select matching features and confirm data types
products = jewelry_toys_df.select('product_id','product_title')
products.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [13]:
# Remove duplicate records and display final product count
products_final = products.dropDuplicates(['product_id'])
products_final.count()

1238134

In [14]:
# Create user dataframe to match customers table
# Select matching feature, obtain count, and rename columns to match table
customers_df = jewelry_toys_df.groupby('customer_ID').count()
customers_df = customers_df.withColumnRenamed("customer_ID","customer_id")
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     195263|             2|
|   46017005|             1|
|   37392783|             5|
|   15650039|            18|
|   39327150|             5|
|   39710436|             2|
|   40617235|             6|
|    1078201|             8|
|   21640906|             6|
|     897990|             1|
|   38389362|            20|
|     726270|             1|
|    2074030|             8|
|   47816064|             1|
|   16200102|             6|
|   18194793|             1|
|   21440635|             2|
|   37398268|             1|
|    2623183|             3|
|    9591866|             1|
+-----------+--------------+
only showing top 20 rows



Postgres Setup

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_hw_db"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [16]:
# Write dataframe to review_id_table in RDS
reviews_final.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# Write dataframe to products table in RDS
products_final.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)